In [73]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [74]:
total_dict = {}
county_dict = {}
state_dict = {}
illinois_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [75]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

1. Read County Data

In [76]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [77]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",165.0,3.0,2.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",117.0,-1.0,4.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1571.0,23.0,184.0,1.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",3.0,0.0,0.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",31.0,0.0,0.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [78]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [79]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))



1.1 Get Constant Classes from today's data

In [80]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [81]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [82]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [83]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '3.0,15.0,42.0,111.0,377.0,212122.0',
    'k': 6},
   'FisherJenks': {'bins': '3048.0,10844.0,23691.0,40947.0,82819.0,212122.0',
    'k': 6},
   'NaturalBreaks': {'bins': '3048.0,10844.0,23691.0,40947.0,82819.0,212122.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '1.3862943611198906,2.772588722239781,3.7612001156935624,4.718498871295094,5.934894195619588,12.264921574089016',
    'k': 6},
   'FisherJenks': {'bins': '1.0986122886681098,2.833213344056216,4.248495242049359,5.652489180268651,7.421775793644647,12.264921574089016',
    'k': 6},
   'NaturalBreaks': {'bins': '1.0986122886681098,2.70805020110221,4.060443010546419,5.429345628954441,7.234898420314831,12.264921574089016',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,3.0,15.0,21356.0', 'k': 6},
   'FisherJenks': {'bins': '139.0,512.0,1100.0,1983.0,3959.0,21356.0', 'k': 6},
   'NaturalBreaks': {'bins': '153.0,529.0,1100.0,1983.0,3959.0,21356.0',
    'k': 6}},
  'log'

1.2 Create Dynamic classes

In [84]:
# case_ts = counties_gdf['cases_ts'].apply(lambda x: x.split(','))
# death_ts = counties_gdf['deaths_ts'].apply(lambda x: x.split(','))

In [85]:
# length = len(case_ts[1])

In [86]:
# new_case_dict = {}
# for i in range(1,length):
#     q6 = mc.Quantiles(case_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

In [87]:
# new_death_dict = {}
# for i in range(1,length):
#     d6 = mc.Quantiles(death_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_death_dict[i] = ','.join(d6.bins.astype(int).astype(str))

In [88]:
# dynamic_county_dict['new_case'] = new_case_dict
# dynamic_county_dict['new_death'] = new_death_dict


2.Read State Data

In [89]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [90]:
state_gdf.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2161.0,17.0,84.0,0.0,2020-01-21,2020-06-08,day,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,2020-03-01,2020-03-06,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",64896.0,966.0,2711.0,12.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",128819.0,749.0,5964.0,25.0,2020-01-21,2020-06-08,day,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",28235.0,338.0,1208.0,11.0,2020-01-21,2020-06-08,day,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",59024.0,431.0,2776.0,27.0,2020-01-21,2020-06-08,day,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [91]:
state_gdf.iloc[34]

NAME                                                  Virgin Islands
population                                                         0
fips                                                              78
dt_first_case                                             2020-03-14
dt_first_death                                            2020-04-06
cases_ts           0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....
deaths_ts          0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....
today_case                                                        71
today_new_case                                                     0
today_death                                                        6
today_new_death                                                    0
dt_start                                                  2020-01-21
dt_end                                                    2020-06-08
dt_unit                                                          day
geometry           (POLYGON ((-64.

In [92]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [93]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [94]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))



In [95]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [96]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

3. Read Illinois County Data

In [97]:
illinois_gdf = gpd.read_file('illinois/nyt_illinois_counties_data.geojson')

In [98]:
illinois_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Clark,Illinois,15836,17023,2020-03-30,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",12.0,1.0,0.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-88.01421 39.48076, -87.96018 ..."
1,Kendall,Illinois,124626,17093,2020-03-18,2020-03-29,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",874.0,5.0,22.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-88.60193 41.71956, -88.40450 ..."
2,Brown,Illinois,6675,17009,2020-05-01,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,0.0,0.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-90.91347 40.10446, -90.69582 ..."
3,Warren,Illinois,17338,17187,2020-04-11,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",127.0,1.0,0.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-90.78519 41.06875, -90.43942 ..."
4,Union,Illinois,17127,17181,2020-04-15,2020-05-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",170.0,0.0,14.0,0.0,2020-01-21,2020-06-08,day,"MULTIPOLYGON (((-89.50179 37.55890, -89.49775 ..."


In [99]:
illinois_case = illinois_gdf['today_case']
illinois_death = illinois_gdf['today_death']
illinois_case_per_100k_capita = illinois_gdf['today_case']/(illinois_gdf['population']/100000)
illinois_death_per_100k_capita = illinois_gdf['today_death']/(illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [100]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [101]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))



In [102]:
illinois_dict['case'] = {}
illinois_dict['death'] = {}
illinois_dict['case_per_100k_capita'] = {}
illinois_dict['death_per_100k_capita'] = {}

In [103]:
illinois_dict['case']['nolog'] = create_dict(illinois_case)
illinois_dict['case']['log'] = create_dict(illinois_log_case)
illinois_dict['death']['nolog'] = create_dict(illinois_death)
illinois_dict['death']['log'] = create_dict(illinois_log_death)
illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

4. Read DPH Illinois County Data

In [104]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [105]:
dph_illinois_gdf.head(5)

,id,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,dt_start,dt_end,dt_unit,population,geometry
0,McHenry,McHenry,"2.0,4.0,6.0,8.0,11.0,12.0,12.0,14.0,19.0,27.0,...",2020-03-17,1733.0,9.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2....",2020-03-26,83.0,0.0,2020-03-17,2020-06-08,day,1,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
1,Boone,Boone,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-05,506.0,2.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-08,18.0,0.0,2020-03-17,2020-06-08,day,1,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
2,Ogle,Ogle,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-31,223.0,2.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-15,3.0,0.0,2020-03-17,2020-06-08,day,1,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
3,Will,Will,"2.0,3.0,9.0,10.0,12.0,21.0,21.0,28.0,40.0,67.0...",2020-03-17,5917.0,27.0,"0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4....",2020-03-20,288.0,0.0,2020-03-17,2020-06-08,day,1,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
4,LaSalle,LaSalle,"0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3....",2020-03-19,166.0,2.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-29,16.0,0.0,2020-03-17,2020-06-08,day,1,"POLYGON ((-88.93885 41.62837, -88.93891 41.628..."


In [106]:
dph_illinois_case = dph_illinois_gdf['today_case']

#illinois_case_per_100k_capita = illinois_gdf['today_case']/(illinois_gdf['population']/100000)
#illinois_death_per_100k_capita = illinois_gdf['today_death']/(illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [107]:
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [108]:
dph_illinois_log_case = dph_illinois_case.apply(lambda x: np.log(x + 1))
# illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
# illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
# illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [109]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [110]:
dph_illinois_dict['case']['nolog'] = create_dict(dph_illinois_case)
dph_illinois_dict['case']['log'] = create_dict(dph_illinois_log_case)
# dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
# dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
# dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
# dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

6. Read DPH County Test (Static) and DPH Zipcode Case (Static)


In [111]:
dph_illinois_test_gdf = gpd.read_file('illinois/dph_county_static_data.geojson')
dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [112]:
dph_zipcode_gdf

,id,confirmed_cases,total_tested,geometry
0,62359,0.0,0.0,"POLYGON ((-91.18290 40.02688, -91.17656 40.020..."
1,62376,0.0,0.0,"POLYGON ((-91.35721 40.17576, -91.35600 40.151..."
2,62312,0.0,0.0,"POLYGON ((-90.97797 39.78881, -90.97787 39.772..."
3,62365,0.0,0.0,"POLYGON ((-91.16584 39.83701, -91.14703 39.837..."
4,62246,7.0,262.0,"POLYGON ((-89.56254 38.91811, -89.56794 38.923..."
...,...,...,...,...
1379,60484,110.0,687.0,"POLYGON ((-87.70969 41.45644, -87.70949 41.456..."
1380,62948,15.0,539.0,"MULTIPOLYGON (((-89.02159 37.74516, -89.03561 ..."
1381,62921,0.0,0.0,"POLYGON ((-89.09155 37.79258, -89.08896 37.807..."
1382,62915,0.0,0.0,"POLYGON ((-89.11994 37.78039, -89.12263 37.781..."


In [113]:
dph_illinois_tested = dph_illinois_test_gdf['total_tested']
dph_zipcode_tested = dph_zipcode_gdf['total_tested']
dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [114]:
dph_illinois_log_tested = dph_illinois_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [115]:
dph_illinois_dict['tested'] = {}
dph_illinois_dict['zipcode_case'] = {}
dph_illinois_dict['zipcode_tested'] = {}

In [116]:
dph_illinois_dict['tested']['nolog'] = create_dict(dph_illinois_tested)
dph_illinois_dict['tested']['log'] = create_dict(dph_illinois_log_tested)
dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

7. Read Worldwide Data

In [117]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

In [118]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']

In [119]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))

In [120]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}

In [121]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)

8. Read Vulnerability Data

In [122]:
vul_gdf = gpd.read_file('illinois/vulnerability.geojson')

In [123]:
vul = vul_gdf['today_vul']

In [124]:
vul_dict['case'] = {}

In [125]:
vul_dict['case']['nolog'] = create_dict(vul)

Summary and ouput


In [126]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['illinois'] = illinois_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_dict, json_file)
print('done')

done
